<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pennylane

In [6]:
import pennylane as qml
import numpy as np

No Exponential Magic

In [7]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def naive_circuit():
    """Create a uniform superposition and return the probabilities.

    Returns:
        array[float]: Probabilities for observing different outcomes.
    """
    for wire in range(n_bits):

        qml.Hadamard(wires=wire)

    return qml.probs(wires=range(n_bits))

The Magic 8-Ball

The matrix form of the oracle

In [4]:
def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2]*len(combo)) # Index of solution
    my_array = np.identity(2**len(combo)) # Create the identity matrix

    test=np.zeros(shape=(2**len(combo)))
    test[index]=1
    my_array = my_array - 2*np.outer(test,test)

    return my_array

In [8]:
combo =[1 ,0 ,0]
oracle_matrix(combo)

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

The oracle in the circuit

In [9]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def oracle_circuit(combo):
    """Create a uniform superposition, apply the oracle, and return probabilities.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        list[float]: The output probabilities.
    """
    for _ in range(n_bits):
        qml.Hadamard(wires= _)

    qml.QubitUnitary(oracle_matrix(combo), wires=[i for i in range(n_bits)])

    return qml.probs(wires=range(n_bits))

In [10]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def pair_circuit(x_tilde, combo):
    """Test a pair labelled by x_tilde for the presence of a solution.

    Args:
        x_tilde (list[int]): An (n_bits - 1)-string labelling the pair to test.
        combo (list[int]): A secret combination of n_bits 0s and 1s.

    Returns:
        array[float]: Probabilities on the last qubit.
    """
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)
    qml.Hadamard(wires=n_bits-1)
    qml.QubitUnitary(oracle_matrix(combo), wires = [i for i in range(n_bits)])
    qml.Hadamard(wires=n_bits-1)

    return qml.probs(wires=n_bits-1)

Breaking the lock a bit faster

In [19]:

def pair_lock_picker(trials):
    """Create a combo, run pair_circuit until it succeeds, and tally success rate.

    Args:
        trials (int): Number of times to test the lock picker.

    Returns:
        float: The average number of times the lock picker uses pair_circuit.
    """
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]

    test_numbers = []

    for trial in range(trials):
        combo = secret_combo(n_bits) # Random list of bits
        counter = 0
        for x_tilde in x_tildes:
            counter += 1

            pair = pair_circuit(x_tilde, combo)
            if np.isclose (1, pair[1]):
                break

        test_numbers.append(counter)
    return sum(test_numbers)/trials

trials = 500
output = pair_lock_picker(trials)

print(f"For {n_bits} bits, it takes", output, "pair tests on average.")


TypeError: object of type 'NoneType' has no len()